# TensorFlow Training and Serving in SageMaker "Script Mode"

Script mode is a training script format for TensorFlow that lets you execute any TensorFlow training script in SageMaker with minimal modification. The [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) handles transferring your script to a SageMaker training instance. On the training instance, SageMaker's native TensorFlow support sets up training-related environment variables and executes your training script. In this tutorial, we use the SageMaker Python SDK to launch a training job and deploy the trained model.

Script mode supports training with a Python script, a Python module, or a shell script. In this example, we use a Python script to train a classification model on the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). In this example, we will show how easily you can train a SageMaker using TensorFlow 1.x and TensorFlow 2.x scripts with SageMaker Python SDK. In addition, this notebook demonstrates how to perform real time inference with the [SageMaker TensorFlow Serving container](https://github.com/aws/sagemaker-tensorflow-serving-container). The TensorFlow Serving container is the default inference method for script mode. For full documentation on the TensorFlow Serving container, please visit [here](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/tensorflow/deploying_tensorflow_serving.rst).


## Set up the environment

Let's start by setting up the environment:

### Install TensorFlow and SageMaker
_Note:  Ignore Warnings and Errors Below_

In [ ]:
!pip uninstall -y tensorflow tensorflow-estimator tb-nightly tf-estimator-nightly # tensorboard

In [ ]:
!pip3 uninstall -y tensorflow tensorflow-estimator tb-nightly tf-estimator-nightly # tensorboard

In [ ]:
!pip3 install tensorflow==2.0.0 --upgrade --ignore-installed --no-cache --user # tensorboard==1.14.0

In [ ]:
!pip3 install sagemaker --upgrade --ignore-installed --no-cache --user

In [ ]:
!pip3 install requests==2.20.1 --user

### Restart the Kernel to Recognize New Dependencies Above

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>", raw=True)

In [ ]:
!pip3 list

## Create the SageMaker Session

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

## Setup the Service Execution Role and Region
Get IAM role arn used to give training and hosting access to your data.  See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).

In [ ]:
role = get_execution_role()
print('RoleARN:  {}\n'.format(role))

region = sagemaker_session.boto_session.region_name
print('Region:  {}'.format(region))

## Training Data

The MNIST dataset has been loaded to the public S3 buckets ``sagemaker-sample-data-<REGION>`` under the prefix ``tensorflow/mnist``.

In [ ]:
original_training_data_uri = 's3://sagemaker-sample-data-{}/tensorflow/mnist'.format(region)
print(original_training_data_uri)

### Copy the Training Data to Your Notebook Disk

In [ ]:
local_data_path = './data'

In [ ]:
!aws --region {region} s3 cp --recursive {original_training_data_uri} {local_data_path}

There are four ``.npy`` file under this prefix:
* ``train_data.npy``
* ``eval_data.npy``
* ``train_labels.npy``
* ``eval_labels.npy``

In [ ]:
!ls {local_data_path}

### Upload the Data to S3 for Distributed Training Across Many Workers
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.

This is S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.

In [ ]:
bucket = sagemaker_session.default_bucket()
data_prefix = 'sagemaker/tensorflow-mnist/data'

In [ ]:
training_data_uri = sagemaker_session.upload_data(path=local_data_path, bucket=bucket, key_prefix=data_prefix)
print(training_data_uri)

In [ ]:
!aws s3 ls --recursive {training_data_uri}

## Train
https://sagemaker.readthedocs.io/en/stable/using_tf.html#distributed-training

This tutorial's training script was adapted from TensorFlow's official [CNN MNIST example](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/layers/cnn_mnist.py). We have modified it to handle the ``model_dir`` parameter passed in by SageMaker. This is an S3 path which can be used for data sharing during distributed training and checkpointing and/or model persistence. We have also added an argument-parsing function to handle processing training-related variables.

At the end of the training job we have added a step to export the trained model to the path stored in the environment variable ``SM_MODEL_DIR``, which always points to ``/opt/ml/model``. This is critical because SageMaker uploads all the model artifacts in this folder to S3 at end of training.

### Training Script

In [ ]:
!ls ./src/mnist_keras_tf2.py

You can add custom Python modules to the `src/requirements.txt` file.  They will automatically be installed - and made available to your training script.

In [ ]:
!cat ./src/requirements.txt

### Train with SageMaker `TensorFlow` Estimator

https://sagemaker.readthedocs.io/en/stable/using_tf.html#distributed-training

The `sagemaker.tensorflow.TensorFlow` estimator handles locating the script mode container, uploading your script to a S3 location and creating a SageMaker training job. Let's call out a couple important parameters here:

* `py_version` is set to `'py3'`
* `script_mode` is set to `True`
* `distributions` is used to configure the distributed training setup.  It's required only if you are doing distributed training either across a cluster of instances or across multiple GPUs.  Here we are using parameter servers as the distributed training schema. SageMaker training jobs run on homogeneous clusters.  To make parameter server more performant in the SageMaker setup, we run a parameter server on every instance in the cluster, so there is no need to specify the number of parameter servers to launch.  You can find the full documentation on how to configure `distributions` [here](https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker/tensorflow#distributed-training).

Notes:  
* This example uses two(2) `ml.p3.2xlarge` instances.  You will likely need to request a SageMaker instance limit increase from Support before continuing.

* Alternatively, you can specify `ml.c5.2xlarge` or choose only one(1) `ml.p3.2xlarge`.

* To recognize the `requirements.txt`, we must include `src/setup.py` per [this](https://github.com/aws/sagemaker-python-sdk/issues/911) GitHub issue.

In [ ]:
from sagemaker.tensorflow import TensorFlow

model_output_path = 's3://{}/sagemaker/tensorflow-mnist/training-runs'.format(bucket)

mnist_estimator = TensorFlow(entry_point='mnist_keras_tf2.py',
                             source_dir='./src',
                             output_path=model_output_path,
                             role=role,
                             train_instance_count=1,
                             train_instance_type='ml.c5.2xlarge',
#                             train_instance_type='local',
                             framework_version='2.0.0',
                             py_version='py3',
                             enable_sagemaker_metrics=True,
                             script_mode=True,
                             distributions={'parameter_server': {'enabled': True}},
                             tags = [{'Key' : 'Project', 'Value' : 'tensorflow-mnist'},
                                     {'Key' : 'TensorBoard', 'Value' : 'dist'}],
                             # Assuming the loglines from the TensorFlow training job are as follows:
                             #    Test loss    : 0.0635053280624561
                             #    Test accuracy: 0.9821
                             metric_definitions=[
                                 {'Name': 'test:loss', 'Regex': 'Test loss    : ([0-9\\.]+)'},
                                 {'Name': 'test:accuracy', 'Regex': 'Test accuracy: ([0-9\\.]+)'},
                             ])

### `fit` the Model (Approx. 15 mins)

To start a training job, we call `estimator.fit(training_data_uri)`.

An S3 location is used here as the input. `fit` creates a default channel named `'training'`, which points to this S3 location. In the training script we can then access the training data from the location stored in `SM_CHANNEL_TRAINING`. `fit` accepts other parameters, as well. See the API doc [here](https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.EstimatorBase.fit) for details.

In [ ]:
mnist_estimator.fit(inputs=training_data_uri, wait=False)

training_job_name = mnist_estimator.latest_training_job.name
print('training_job_name:  {}'.format(training_job_name))

After some time, or in a separate Python notebook, we can attach to the running job using the `training_job_name`.

In [ ]:
from sagemaker.tensorflow import TensorFlow

mnist_estimator = TensorFlow.attach(training_job_name=training_job_name)

In [ ]:
!aws --region {region} s3 ls --recursive {model_output_path}/{training_job_name}

In [ ]:
print(model_output_path)

## Option 1:  Predict Directly in the Notebook

Use TensorFlow Core to load the model from `model_output_path`

In [ ]:
!aws --region {region} s3 ls --recursive {model_output_path}/{training_job_name}/output/

In [ ]:
!aws --region {region} s3 cp {model_output_path}/{training_job_name}/output/model.tar.gz ./model/model.tar.gz

In [ ]:
saved_model_path = './model/000000001'

In [ ]:
!rm -rf {saved_model_path}

In [ ]:
!ls ./model

In [ ]:
!tar -xzvf ./model/model.tar.gz -C ./model

Load the model and list the prediction signatures

In [ ]:
import tensorflow as tf

loaded_model = tf.saved_model.load(export_dir='./model/000000001/')

loaded_model.signatures

Show the prediction signature details

In [ ]:
from tensorflow.python.tools import saved_model_cli

parser = saved_model_cli.create_parser()
args = parser.parse_args(['show', '--dir', saved_model_path, '--tag_set', 'serve', '--signature_def', 'serving_default'])
saved_model_cli.show(args)

### Perform inference with the signature

In [ ]:
import numpy as np

train_data = np.load('{}/train_data.npy'.format(local_data_path))
train_labels = np.load('{}/train_labels.npy'.format(local_data_path))

eval_data = np.load('{}/eval_data.npy'.format(local_data_path))
eval_labels = np.load('{}/eval_labels.npy'.format(local_data_path))

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

image_idx = 4444 # random idx number

plt.imshow(eval_data[image_idx].reshape(28, 28), cmap='Greys')

In [ ]:
predict_fn = loaded_model.signatures["serving_default"]

result = predict_fn(input_1=tf.constant(eval_data[image_idx].reshape(1, 28, 28, 1)))
prediction = result['output_1'].numpy().argmax()

label = eval_labels[image_idx]
print('Prediction is {}, Label is {}, Matched: {}'.format(prediction, label, prediction == label))

## Option 2:  Create a SageMaker Endpoint and Perform REST-based Predictions

### Deploy the Trained Model to a SageMaker Endpoint (Approx. 10 mins)

After training, we use the `TensorFlow` estimator object to build and deploy a `TensorFlowPredictor`.  This creates a Sagemaker Endpoint -- a hosted prediction service that we can use to perform inference.

As mentioned above we have implementation of `model_fn` in the `tensorflow_mnist.py` script that is required. We are going to use default implementations of `input_fn`, `predict_fn`, `output_fn` and `transform_fm` defined in [sagemaker-tensorflow-containers](https://github.com/aws/sagemaker-tensorflow-containers).

The `deploy()` method creates a SageMaker model, which is then deployed to an endpoint to serve prediction requests in real time.  We will use the TensorFlow Serving container for the endpoint, because we trained with script mode.  This serving container runs an implementation of a web server that is compatible with SageMaker hosting protocol.  The [Using your own inference code](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-main.html) document explains how SageMaker runs inference containers.

The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint.  These do not need to be the same as the values we used for the training job.  For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances, but you need to make sure that you return or save your model as a cpu model similar to what we did in `mnist.py`.  Here we will deploy the model to a single `ml.p3.2xlarge` instance.  Alternatively, you can use a `ml.c5.2xlarge` instance.

In [ ]:
predictor = mnist_estimator.deploy(initial_instance_count=1, instance_type='ml.c5.2xlarge')

### Invoke the Endpoint

Let's download the training data and use that as input for inference.

In [ ]:
import numpy as np

train_data = np.load('{}/train_data.npy'.format(local_data_path))
train_labels = np.load('{}/train_labels.npy'.format(local_data_path))

The formats of the input and the output data correspond directly to the request and response formats of the `Predict` method in the [TensorFlow Serving REST API](https://www.tensorflow.org/serving/api_rest). SageMaker's TensforFlow Serving endpoints can also accept additional input formats that are not part of the TensorFlow REST API, including the simplified JSON format, line-delimited JSON objects ("jsons" or "jsonlines"), and CSV data.

In this example we are using a `numpy` array as input, which will be serialized into the simplified JSON format. In addition, TensorFlow Serving can also process multiple items at once as you can see in the following code. You can find the complete documentation on how to make predictions against a TensorFlow Serving SageMaker endpoint [here](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/tensorflow/deploying_tensorflow_serving.rst#making-predictions-against-a-sagemaker-endpoint).

Examine the prediction result from the TensorFlow 2.0 model.

In [ ]:
predictions = predictor.predict(train_data[:50])
for i in range(0, 50):
    prediction = predictions['predictions'][i]
    label = train_labels[i]
    print('Prediction is {}, Label is {}, Matched: {}'.format(prediction, label, prediction == label))

### (Optional) Cleanup Endpoint

Let's delete the endpoint we just created to prevent incurring any extra costs.

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)